In [1]:
import os

In [2]:
# exceute my project working directoy
%pwd

'c:\\Users\\Farah\\python\\cassandra.HBase.pig.Spark MLlib\\End To End MLOPS Data Science Project Implementation With Deployment\\MLflow-Project\\research'

In [3]:
# khsni n goback lfolder principal dyali
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Farah\\python\\cassandra.HBase.pig.Spark MLlib\\End To End MLOPS Data Science Project Implementation With Deployment\\MLflow-Project'

# hada dyal entuty

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
!pip install python-box

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 1.2/1.2 MB 279.4 kB/s eta 0:00:00


In [11]:
!pip install ensure

Defaulting to user installation because normal site-packages is not writeable
  Using cached ensure-1.0.4-py3-none-any.whl (15 kB)


In [12]:
from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml,create_directories

# hadi configurarion manager

In [13]:
from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml,create_directories
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

# hadi componentfile

In [16]:
import os
import urllib.request as request
import zipfile
from src.mlproject import logger
from src.mlproject.utils.common import get_size

In [17]:
import os
import urllib.request as request
import zipfile
from src.mlproject import logger
from src.mlproject.utils.common import get_size
class DataIngestion:#atchd configuartion w t3tini path
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):# hada ghadi ydownloadi data
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):#khs extract had zip file
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

# daba pipeline

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-01 22:23:59,317:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-06-01 22:23:59,324:INFO:common:yaml file: params.yaml loaded successfully]
[2024-06-01 22:23:59,328:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-06-01 22:23:59,332:INFO:common:created directory at: artifacts]
[2024-06-01 22:23:59,335:INFO:common:created directory at: artifacts/data_ingestion]
[2024-06-01 22:24:00,242:INFO:2280763589:artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 10BA:31C0C8:73E972:7AF22D:665B742C
Accept-Ranges: bytes
Date: Sat, 01 Jun 2024 19:19:10